In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !unzip /content/drive/MyDrive/CheXpert-v1.0-small.zip

In [3]:
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
def load_data(csv):
    chunks = pd.read_csv(csv, usecols=['Path', 'Pneumonia'], chunksize=100)
    dfs = []
    for df in chunks:
        df = df[df['Pneumonia'].notna()]
        df = df[df['Pneumonia'] != -1]
        dfs.append(df)
    return pd.concat(dfs)


def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    return img


def extract_data(df):
    for index, row in df.iterrows():
        img = load_image(row['Path'])
        label = row['Pneumonia']
        yield img, label


# Load data from csv files in chunks
train_df_chunks = load_data("/content/CheXpert-v1.0-small/train.csv")
valid_df_chunks = load_data("/content/CheXpert-v1.0-small/valid.csv")

In [5]:
train_df_chunks['Pneumonia'].value_counts(), valid_df_chunks['Pneumonia'].value_counts()

(1.0    6039
 0.0    2799
 Name: Pneumonia, dtype: int64, 0.0    226
 1.0      8
 Name: Pneumonia, dtype: int64)

In [6]:
# Define generators for train and valid data
train_gen = extract_data(train_df_chunks)
valid_gen = extract_data(valid_df_chunks)

# Extract data and labels using generators
train_data, train_labels = zip(*train_gen)
valid_data, valid_labels = zip(*valid_gen)

# Convert lists to numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)

In [7]:
def augment_data(data, labels, batch_size=1):
    # Calculate number of zeros and ones
    zeros = np.count_nonzero(labels == 0)
    ones = np.count_nonzero(labels == 1)
    # Calculate difference between zeros and ones
    diff = abs(zeros - ones)
    # Create ImageDataGenerator object for augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode="nearest"
    )
    if zeros > ones:
        # Augment ones to balance classes
        augment_indices = np.random.choice(np.where(labels == 1)[0], diff)
        new_data = np.empty((diff, ) + data.shape[1:], dtype=data.dtype)
        new_labels = np.ones(diff, dtype=labels.dtype)
        for i, index in enumerate(augment_indices):
            # Reshape image to 4D tensor for augmentation
            img = data[index].reshape((1, ) + data[index].shape)
            # Generate augmented images
            aug_imgs = datagen.flow(img, batch_size=batch_size)
            # Append augmented images to data list
            new_data[i:i+batch_size] = aug_imgs[0]
        # Concatenate augmented data and labels with original data and labels
        data = np.concatenate((data, new_data), axis=0)
        labels = np.concatenate((labels, new_labels), axis=0)
    elif ones > zeros:
        # Augment zeros to balance classes
        augment_indices = np.random.choice(np.where(labels == 0)[0], diff)
        new_data = np.empty((diff, ) + data.shape[1:], dtype=data.dtype)
        new_labels = np.zeros(diff, dtype=labels.dtype)
        for i, index in enumerate(augment_indices):
            # Reshape image to 4D tensor for augmentation
            img = data[index].reshape((1, ) + data[index].shape)
            # Generate augmented images
            aug_imgs = datagen.flow(img, batch_size=batch_size)
            # Append augmented images to data list
            new_data[i:i+batch_size] = aug_imgs[0]
        # Concatenate augmented data and labels with original data and labels
        data = np.concatenate((data, new_data), axis=0)
        labels = np.concatenate((labels, new_labels), axis=0)
    return data, labels

# Augment train data and labels with batch size of 16
train_data, train_labels = augment_data(train_data, train_labels, batch_size=16)
valid_data, valid_labels = augment_data(valid_data, valid_labels, batch_size=16)

In [8]:
def shuffle_data(data, labels):
    # Get shuffled indices
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    # Shuffle data and labels using shuffled indices
    data = data[indices]
    labels = labels[indices]
    return data, labels

# Shuffle train data and labels
train_data, train_labels = shuffle_data(train_data, train_labels)
valid_data, valid_labels = shuffle_data(valid_data, valid_labels)

In [9]:
def check_balance(arr):
    num_zeros = sum(arr == 0)
    num_ones = sum(arr == 1)
    if num_zeros == num_ones:
        return True, num_zeros, num_ones
    else:
        return False, num_zeros, num_ones

# Check if training labels are balanced after augmentation
train_balanced = check_balance(train_labels)
print("Training data is balanced:", train_balanced)

# Check if validation labels are balanced after augmentation
valid_balanced = check_balance(valid_labels)
print("Validation data is balanced:", valid_balanced)

Training data is balanced: (True, 6039, 6039)
Validation data is balanced: (True, 226, 226)


In [10]:
train_data_nor = np.empty(train_data.shape, dtype=np.float32)
for i in range(train_data.shape[0]):
    train_data_nor[i] = train_data[i].astype(np.float32) / 255.0

In [11]:
valid_data_nor = np.empty(valid_data.shape, dtype=np.float32)
for i in range(valid_data.shape[0]):
    valid_data_nor[i] = valid_data[i].astype(np.float32) / 255.0

In [12]:
# !pip install vit_keras
# !pip install tensorflow_addons

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Reshape, GlobalAveragePooling2D, GlobalMaxPooling2D 
from tensorflow.keras.optimizers import Adam
from vit_keras import vit

# Define CNN 1 layers
cnn_input1 = Input(shape=(224, 224, 3))
cnn_layer1 = Conv2D(filters=32, kernel_size=3, activation='relu')(cnn_input1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Conv2D(filters=64, kernel_size=3, activation='relu')(cnn_layer1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Conv2D(filters=128, kernel_size=3, activation='relu')(cnn_layer1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)

# Define CNN 2 layers
cnn_input2 = Input(shape=(224, 224, 3))
cnn_layer2 = Conv2D(filters=32, kernel_size=3, activation='relu')(cnn_input2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Conv2D(filters=64, kernel_size=3, activation='relu')(cnn_layer2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Conv2D(filters=128, kernel_size=3, activation='relu')(cnn_layer2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)

# Concatenate CNN layers
concat_layer = Concatenate()([cnn_layer1, cnn_layer2])

# Pooling layers
pooled_resnet = GlobalMaxPooling2D()(concat_layer)

# Define ViT layers
vit_input1 = Input(shape=(224, 224, 3))
vit_layer1 = vit.vit_b16(
    image_size=224,
    classes = 2,
    activation='sigmoid',
    pretrained=True,
    include_top=True
)(vit_input1)
vit_output1 = Reshape((256,))(pooled_resnet)
flatten = Flatten()(vit_output1)
output = Dense(units=1, activation='sigmoid')(flatten)

# Define the model
model = tf.keras.Model(inputs=[cnn_input1, cnn_input2, vit_input1], outputs=output)
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.9/dist-packages/vit_keras/vit.py:139: UserWarning: Can only use pretrained_top with imagenet21k+imagenet2012 if classes = 1000. Setting manually.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 222, 222, 32  896         ['input_1[0][0]']                
                                )                                                             

In [14]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit([train_data, train_data, train_data], train_labels, epochs=10, batch_size=4, 
                    validation_data=([valid_data, valid_data, valid_data], valid_labels))